In [3]:
import pandas as pd
import datetime
def Time2Str(tsm):
    t1 = datetime.datetime.fromisoformat(tsm)
    t0 = t1-datetime.timedelta(days=0, hours=1)
    t2 = t1+datetime.timedelta(days=0, hours=1)
    str1 = t0.strftime("%Y%m%d")+'(' +str(t0.hour).rjust(2,'0')+'#'+str(t1.hour).rjust(2,'0')+')'
    str2 = t1.strftime("%Y%m%d")+'(' +str(t1.hour).rjust(2,'0')+'#'+str(t2.hour).rjust(2,'0')+')'
    return str1+';'+str2


In [5]:
Time2Str('2021-11-16 15:51:39')

'20211116(14#15);20211116(15#16)'

In [4]:
df = pd.DataFrame({
    'Buy':['BUY_03','BUY_02','BUY_01','BUY_04','BUY_03','BUY_02','BUY_01','BUY_04'],
    'Times':['2021-11-16 00:03:32','2021-11-16 00:12:23','2021-11-16 00:22:07','2021-11-16 21:10:24',\
    '2021-11-16 21:18:05','2021-11-16 21:22:02','2021-11-16 21:42:57','2021-11-16 23:51:39'],
    'Seller':['Y','Y','Y','E','E','E','E','Y']
    })


In [6]:
# 时间离散化
df['tsm'] = df['Times'].apply(Time2Str)

In [7]:
df

,Buy,Times,Seller,tsm
0,BUY_03,2021-11-16 00:03:32,Y,20211115(23#00);20211116(00#01)
1,BUY_02,2021-11-16 00:12:23,Y,20211115(23#00);20211116(00#01)
2,BUY_01,2021-11-16 00:22:07,Y,20211115(23#00);20211116(00#01)
3,BUY_04,2021-11-16 21:10:24,E,20211116(20#21);20211116(21#22)
4,BUY_03,2021-11-16 21:18:05,E,20211116(20#21);20211116(21#22)
5,BUY_02,2021-11-16 21:22:02,E,20211116(20#21);20211116(21#22)
6,BUY_01,2021-11-16 21:42:57,E,20211116(20#21);20211116(21#22)
7,BUY_04,2021-11-16 23:51:39,Y,20211116(22#23);20211116(23#00)


In [13]:
df = df.set_index(["Buy", "Times",'Seller'])["tsm"].str.split(";", expand=True)\
    .stack().reset_index(drop=True, level=-1).reset_index().rename(columns={0: "tsm"})
print(df)

       Buy                Times Seller              tsm
0   BUY_03  2021-11-16 00:03:32      Y  20211115(23#00)
1   BUY_03  2021-11-16 00:03:32      Y  20211116(00#01)
2   BUY_02  2021-11-16 00:12:23      Y  20211115(23#00)
3   BUY_02  2021-11-16 00:12:23      Y  20211116(00#01)
4   BUY_01  2021-11-16 00:22:07      Y  20211115(23#00)
5   BUY_01  2021-11-16 00:22:07      Y  20211116(00#01)
6   BUY_04  2021-11-16 21:10:24      E  20211116(20#21)
7   BUY_04  2021-11-16 21:10:24      E  20211116(21#22)
8   BUY_03  2021-11-16 21:18:05      E  20211116(20#21)
9   BUY_03  2021-11-16 21:18:05      E  20211116(21#22)
10  BUY_02  2021-11-16 21:22:02      E  20211116(20#21)
11  BUY_02  2021-11-16 21:22:02      E  20211116(21#22)
12  BUY_01  2021-11-16 21:42:57      E  20211116(20#21)
13  BUY_01  2021-11-16 21:42:57      E  20211116(21#22)
14  BUY_04  2021-11-16 23:51:39      Y  20211116(22#23)
15  BUY_04  2021-11-16 23:51:39      Y  20211116(23#00)


In [19]:
df_0 = pd.merge(df,df,on =['Seller','tsm'],how='inner')
df_1 = df_0[df_0['Buy_x']!=df_0['Buy_y']]
df_1['diff'] = (pd.to_datetime(df_0['Times_x'])-\
pd.to_datetime(df_0['Times_y'])).dt.seconds/3600/24

<ipython-input-19-3d6d32a84a4f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['diff'] = (pd.to_datetime(df_0['Times_x'])-\


In [23]:
df_1['diff']

1     0.993854
2     0.987095
3     0.006146
5     0.993241
6     0.012905
7     0.006759
10    0.993854
11    0.987095
12    0.006146
14    0.993241
15    0.012905
16    0.006759
19    0.994664
20    0.991921
21    0.977396
22    0.005336
24    0.997257
25    0.982731
26    0.008079
27    0.002743
29    0.985475
30    0.022604
31    0.017269
32    0.014525
35    0.994664
36    0.991921
37    0.977396
38    0.005336
40    0.997257
41    0.982731
42    0.008079
43    0.002743
45    0.985475
46    0.022604
47    0.017269
48    0.014525
Name: diff, dtype: float64

In [24]:
df_1.groupby(['Buy_x','Buy_y']).agg({'Seller': pd.Series.nunique}).reset_index()

,Buy_x,Buy_y,Seller
0,BUY_01,BUY_02,2
1,BUY_01,BUY_03,2
2,BUY_01,BUY_04,1
3,BUY_02,BUY_01,2
4,BUY_02,BUY_03,2
5,BUY_02,BUY_04,1
6,BUY_03,BUY_01,2
7,BUY_03,BUY_02,2
8,BUY_03,BUY_04,1
9,BUY_04,BUY_01,1


# 接下来再分别计算X的总次数Y的总次数，XY的并集和交集，再计算一下相似度。如果相似度非常高那么是欺诈的可能性很大